In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

cctv_seoul = pd.read_csv(r'./data/CCTV.csv', encoding='cp949')
pop_seoul  = pd.read_csv('.\data\등록인구.csv', encoding='utf-8')


<>:7: SyntaxWarning: invalid escape sequence '\d'
<>:7: SyntaxWarning: invalid escape sequence '\d'
C:\Users\paint\AppData\Local\Temp\ipykernel_1756\753777552.py:7: SyntaxWarning: invalid escape sequence '\d'
  pop_seoul  = pd.read_csv('.\data\등록인구.csv', encoding='utf-8')


파일 위치 앞의 . 개수가 몇개의 상위 경로로 갈 지를 뜻함 ex) r'../데이터시각화/data~'

In [18]:
cctv_seoul.head()

,자치구,안심 주소,위도,경도,CCTV 수량,수정 일시
0,중랑구,중화2동 322-16(고정2)_C-JH02-0003-B,37.5963,127.0739,1,2022-12-01
1,중랑구,신내2동 651(고정4)_929,37.6150,127.0889,1,2022-12-01
2,중랑구,중화2동 323-14 (초롱공원)_C-JH02-0023,37.5957,127.0732,1,2022-12-01
3,중랑구,신내2동 602-1 (시조목 관리 보조)_C-SN02-0001-A,37.6066,127.0919,1,2022-12-01
4,중랑구,중화2동 323-14(고정1)_C-JH02-0023-A,37.5957,127.0732,1,2022-12-01


안심 주소 언더바 뒤에 cctv의 id가 들어있음

In [19]:
cctv_seoul.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83734 entries, 0 to 83733
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   자치구      83734 non-null  object 
 1   안심 주소    83734 non-null  object 
 2   위도       83734 non-null  float64
 3   경도       83734 non-null  float64
 4   CCTV 수량  83734 non-null  int64  
 5   수정 일시    83734 non-null  object 
dtypes: float64(2), int64(1), object(3)
memory usage: 3.8+ MB


In [20]:
data=pd.DataFrame()
data['구별']=cctv_seoul['자치구'].unique()
cctv_count=cctv_seoul.groupby('자치구').sum(['CCTV 수량'])
cctv_count.reset_index(inplace=True)
cctv_count.drop(['위도', '경도'], axis=1, inplace=True)
cctv_count=cctv_count.rename(columns={'자치구':'구별'})
data=pd.merge(data,cctv_count,on='구별')
data.head()

,구별,CCTV 수량
0,중랑구,3471
1,중구,2336
2,종로구,1930
3,은평구,4116
4,용산구,1078


In [21]:
cctv_gu_count=cctv_seoul.groupby('자치구').sum()['CCTV 수량'] #시리즈로 저장됨
cctv_gu=pd.DataFrame(cctv_gu_count) #데이터프레임으로 변환
cctv_gu=cctv_gu.reset_index() #인덱스를 컬럼으로 변환
cctv_gu=cctv_gu.rename(columns={'자치구':'구별'}) #컬럼명 변경

인덱스한테도 이름이 있다. 고로 멀티 인덱스 가능     
rename 할 때 inplace=True 옵션은 사용하지 말고 재정의 방법을 더 추천함      
method chaining을 사용하기 어려워지기 때문

In [22]:
pop_seoul.head(10)

,동별(1),항목,2025 2/4
0,동별(1),항목,합계
1,합계,계,9595509
2,합계,한국인,9325616
3,합계,등록외국인,269893
4,종로구,계,149332
5,종로구,한국인,138015
6,종로구,등록외국인,11317
7,중구,계,129616
8,중구,한국인,119097
9,중구,등록외국인,10519


iloc: 절대위치(인덱스 번호가 바껴도 위치에 따라 지정 가능)    
loc: 인덱스  

In [23]:
pop_seoul_gu=pop_seoul[pop_seoul['항목']=='계']
pop_seoul_gu.drop('항목',inplace=True, axis=1)
pop_seoul_gu=pop_seoul_gu.rename(columns={'동별(1)':'구별'})
pop_seoul_gu.reset_index(drop=True, inplace=True)
pop_seoul_gu= pop_seoul_gu.drop(index=0).reset_index(drop=True)
pop_seoul_gu=pop_seoul_gu.rename(columns={'2025 2/4':'인구수'})

C:\Users\paint\AppData\Local\Temp\ipykernel_1756\164031717.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pop_seoul_gu.drop('항목',inplace=True, axis=1)


In [24]:
pop_seoul_gu.head()

,구별,인구수
0,종로구,149332
1,중구,129616
2,용산구,216214
3,성동구,280683
4,광진구,350164


In [25]:
data=pd.merge(data,pop_seoul_gu,on='구별')
data.head()

,구별,CCTV 수량,인구수
0,중랑구,3471,383958
1,중구,2336,129616
2,종로구,1930,149332
3,은평구,4116,463165
4,용산구,1078,216214


* 기준시점 : 연간은 매년 12.31일기준, 분기는 매분기 말기준      
* 분기(2025 2/4) 
* 세대수, 세대당인구 : 외국인 제외
* 65세이상 고령자 수 : 외국인 포함

In [26]:
crime_seoul= pd.read_csv(r'./data/범죄현황.csv', encoding='utf-8',
                         skiprows=2,header=[0,1])
#2번째 줄까지 건너뛰고, 3,4번째 줄을 헤더로 지정

In [27]:
crime_seoul.head()

자치구별(1) 자치구별(2)     소계          살인        강도      강간·강제추행           절도  \
  자치구별(1) 자치구별(2)     발생     검거   발생   검거   발생   검거      발생    검거     발생   
0      합계      소계  86213  64823  150  144  129  117    5764  5395  37038   
1      합계     종로구   2981   3485    5    6    5    7     239  1138   1169   
2      합계      중구   3348   2465    3    3    6    5     212   144   1520   
3      합계     용산구   3021   2268    3    4    3    3     245   219    978   
4      합계     성동구   2023   1411    2    1    2    2     119    85    893   

             폭력         
      검거     발생     검거  
0  21600  43132  37567  
1    953   1563   1381  
2    864   1607   1449  
3    561   1792   1481  
4    472   1007    851

In [28]:
crime_seoul.columns

MultiIndex([('자치구별(1)', '자치구별(1)'),
            ('자치구별(2)', '자치구별(2)'),
            (     '소계',      '발생'),
            (     '소계',      '검거'),
            (     '살인',      '발생'),
            (     '살인',      '검거'),
            (     '강도',      '발생'),
            (     '강도',      '검거'),
            ('강간·강제추행',      '발생'),
            ('강간·강제추행',      '검거'),
            (     '절도',      '발생'),
            (     '절도',      '검거'),
            (     '폭력',      '발생'),
            (     '폭력',      '검거')],
           )

📌 이터너블(iterable)이란?

한 번에 하나씩 꺼낼 수 있는 객체를 말합니다.
쉽게 말해, for문에 넣어서 차례차례 반복할 수 있는 모든 것들이 이터너블이에요.

다중 헤더(MulitiIndex)컬럼을 단일 텍스트 컬럼으로 변환      
`crime_seoul.columns`
→ 컬럼명이 MultiIndex(튜플)로 되어 있음. 예: ('2023', '1월')

`.map(lambda x: ...)`
→ 각 컬럼 튜플에 대해 함수를 적용함.        
#map함수: 데이터를 순서대로 가져와서(이터너블한 객체) 하나씩 적용시키는 함수     

`lambda x: '_'.join(x)`
→ 튜플 ('2023','1월')을 받아서 "2023_1월"로 합침.

join은 리스트나 튜플을 하나의 문자열로 묶는 함수예요.

'_'를 사이에 넣어서 "상위_하위" 형태로 컬럼명을 단일 문자열로 바꾸는 겁니다.

In [29]:
crime_seoul.columns=crime_seoul.columns.map(lambda x: '_'.join(x))
#다중 헤더(MulitiIndex)컬럼을 단일 텍스트 컬럼으로 변환
crime_seoul.head()

,자치구별(1)_자치구별(1),자치구별(2)_자치구별(2),소계_발생,소계_검거,살인_발생,살인_검거,강도_발생,강도_검거,강간·강제추행_발생,강간·강제추행_검거,절도_발생,절도_검거,폭력_발생,폭력_검거
0,합계,소계,86213,64823,150,144,129,117,5764,5395,37038,21600,43132,37567
1,합계,종로구,2981,3485,5,6,5,7,239,1138,1169,953,1563,1381
2,합계,중구,3348,2465,3,3,6,5,212,144,1520,864,1607,1449
3,합계,용산구,3021,2268,3,4,3,3,245,219,978,561,1792,1481
4,합계,성동구,2023,1411,2,1,2,2,119,85,893,472,1007,851


In [30]:
crime_seoul=crime_seoul.rename(columns={'자치구별(2)_자치구별(2)':'구별',
                                        '소계_발생':'범죄발생',
                                        '소계_검거':'범죄검거'})
crime_seoul=crime_seoul[['구별','범죄발생','범죄검거']]
crime_seoul=crime_seoul[crime_seoul['구별']!='소계'].reset_index(drop=True)
crime_seoul.head()

,구별,범죄발생,범죄검거
0,종로구,2981,3485
1,중구,3348,2465
2,용산구,3021,2268
3,성동구,2023,1411
4,광진구,3424,2333


In [31]:
data=pd.merge(data,crime_seoul,on='구별')
#merge할 때 left_on, right_on 옵션 사용 가능(첫번째 데이터프레임의 컬럼명, 두번째 데이터프레임의 컬럼명이 다를 때)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   구별       25 non-null     object
 1   CCTV 수량  25 non-null     int64 
 2   인구수      25 non-null     object
 3   범죄발생     25 non-null     int64 
 4   범죄검거     25 non-null     int64 
dtypes: int64(3), object(2)
memory usage: 1.1+ KB


데이터를 병합하는 데에는 concat, merge, join을 사용할 수 있다.      
merge로 join의 기능을 대체 가능하고 더 우수함   

merge할 때 how='left'인 경우 df1에는 없고 df2에만 있는 키인 경우 데이터 사라짐, df2에 없는 키는 결과에 포함되지만, 그 값은 NaN으로 채워짐. >  반대 'left'     
how='inner'인 경우는 두 데이터프레임에 둘 다 존재하는 키일 때만 남김  >  반대 'outer'     

default: inner

파생변수 생성(assign, apply)
- cctv 비율, 인구비율, 검거율 등

In [32]:
data['인구수']=data['인구수'].astype(int)

In [33]:
data=data.assign(
    cctv비율=data['CCTV 수량']/data['CCTV 수량'].sum()*100,
    인구비율=data['인구수']/data['인구수'].sum()*100,
    검거율=data['범죄검거']/data['범죄발생']*100,
    cctv인구비율=data['CCTV 수량']/data['인구수']*100,
    범죄율= data['범죄발생']/data['인구수']*100000
)
data.head()
data.to_csv('./data/통합데이터.csv', index=False)